In [1]:
import sys
sys.path.append('../scripts/')
from puddle_world import *
import itertools 
import collections 
from copy import copy

In [2]:
class DynamicProgramming: 
    def __init__(self, widths, goal, puddles, time_interval, sampling_num, \
                 puddle_coef=100.0, lowerleft=np.array([-4, -4]).T, upperright=np.array([4, 4]).T): 
        self.pose_min = np.r_[lowerleft, 0]
        self.pose_max = np.r_[upperright, math.pi*2]
        self.widths = widths
        self.goal = goal
        
        self.index_nums = ((self.pose_max - self.pose_min)/self.widths).astype(int)
        nx, ny, nt = self.index_nums
        self.indexes = list(itertools.product(range(nx), range(ny), range(nt)))
        
        self.value_function, self.final_state_flags =  self.init_value_function() 
        self.policy = self.init_policy()
        self.actions = list(set([tuple(self.policy[i]) for i in self.indexes]))
        
        self.state_transition_probs = self.init_state_transition_probs(time_interval, sampling_num)
        self.depths = self.depth_means(puddles, sampling_num)
        
        self.time_interval = time_interval
        self.puddle_coef = puddle_coef
        
    def value_iteration_sweep(self): #追加
        max_delta = 0.0
        for index in self.indexes:
            if not self.final_state_flags[index]:
                max_q = -1e100
                max_a = None
                qs = [self.action_value(a, index) for a in self.actions] #全行動の行動価値を計算
                max_q = max(qs)                               #最大の行動価値
                max_a = self.actions[np.argmax(qs)]   #最大の行動価値を与える行動
                
                delta = abs(self.value_function[index] - max_q)            #変化量
                max_delta = delta if delta > max_delta else max_delta #スイープ中で最大の変化量の更新
                
                self.value_function[index] = max_q      #価値の更新
                self.policy[index] = np.array(max_a).T  #方策の更新
            
        return max_delta        
        
    def policy_evaluation_sweep(self):   
        max_delta = 0.0
        for index in self.indexes:
            if not self.final_state_flags[index]:
                q = self.action_value(tuple(self.policy[index]), index)
                
                delta = abs(self.value_function[index] - q)
                max_delta = delta if delta > max_delta else max_delta
                
                self.value_function[index] = q
            
        return max_delta
    
    def action_value(self, action, index): #はみ出しペナルティー追加
        value = 0.0
        for delta, prob in self.state_transition_probs[(action, index[2])]: 
            after, edge_reward = self.edge_correction(np.array(index).T + delta)
            after = tuple(after)
            reward = - self.time_interval * self.depths[(after[0], after[1])] * self.puddle_coef - self.time_interval + edge_reward
            value += (self.value_function[after] + reward) * prob

        return value
            
    def edge_correction(self, index): #変更
        edge_reward = 0.0
        index[2] = (index[2] + self.index_nums[2])%self.index_nums[2] #方角の処理
        
        for i in range(2):
            if index[i] < 0:
                index[i] = 0
                edge_reward = -1e100
            elif index[i] >= self.index_nums[i]:
                index[i] = self.index_nums[i]-1
                edge_reward = -1e100
                
        return index, edge_reward
        
    def depth_means(self, puddles, sampling_num):
        ###セルの中の座標を均等にsampling_num**2点サンプリング###
        dx = np.linspace(0, self.widths[0], sampling_num) 
        dy = np.linspace(0, self.widths[1], sampling_num)
        samples = list(itertools.product(dx, dy))
        
        tmp = np.zeros(self.index_nums[0:2]) #深さの合計が計算されて入る
        for xy in itertools.product(range(self.index_nums[0]), range(self.index_nums[1])):
            for s in samples:
                pose = self.pose_min + self.widths*np.array([xy[0], xy[1], 0]).T + np.array([s[0], s[1], 0]).T #セルの中心の座標
                for p in puddles:
                    tmp[xy] += p.depth*p.inside(pose) #深さに水たまりの中か否か（1 or 0）をかけて足す
                        
            tmp[xy] /= sampling_num**2 #深さの合計から平均値に変換
                       
        return tmp
    
    def init_state_transition_probs(self, time_interval, sampling_num):
        ###セルの中の座標を均等にsampling_num**3点サンプリング###
        dx = np.linspace(0.001, self.widths[0]*0.999, sampling_num) #隣のセルにはみ出さないように端を避ける
        dy = np.linspace(0.001, self.widths[1]*0.999, sampling_num)
        dt = np.linspace(0.001, self.widths[2]*0.999, sampling_num)
        samples = list(itertools.product(dx, dy, dt))
        
        ###各行動、各方角でサンプリングした点を移動してインデックスの増分を記録###
        tmp = {}
        for a in self.actions:
            for i_t in range(self.index_nums[2]):
                transitions = []
                for s in samples:
                    before = np.array([s[0], s[1], s[2] + i_t*self.widths[2]]).T + self.pose_min  #遷移前の姿勢
                    before_index = np.array([0, 0, i_t]).T                                                      #遷移前のインデックス
                
                    after = IdealRobot.state_transition(a[0], a[1], time_interval, before)   #遷移後の姿勢
                    after_index = np.floor((after - self.pose_min)/self.widths).astype(int)   #遷移後のインデックス
                    
                    transitions.append(after_index - before_index)                                  #インデックスの差分を追加
                    
                unique, count = np.unique(transitions, axis=0, return_counts=True)   #集計（どのセルへの遷移が何回か）
                probs = [c/sampling_num**3 for c in count]                   #サンプル数で割って確率にする
                tmp[a,i_t] = list(zip(unique, probs))
                
        return tmp
        
    def init_policy(self):
        tmp = np.zeros(np.r_[self.index_nums,2]) #制御出力が2次元なので、配列の次元を4次元に
        for index in self.indexes:
            center = self.pose_min + self.widths*(np.array(index).T + 0.5)  #セルの中心の座標
            tmp[index] = PuddleIgnoreAgent.policy(center, self.goal)
            
        return tmp
        
    def init_value_function(self): 
        v = np.empty(self.index_nums) #全離散状態を要素に持つ配列を作成
        f = np.zeros(self.index_nums) 
        
        for index in self.indexes:
            f[index] = self.final_state(np.array(index).T)
            v[index] = self.goal.value if f[index] else -100.0
                
        return v, f
        
    def final_state(self, index):
        x_min, y_min, _ = self.pose_min + self.widths*index          #xy平面で左下の座標
        x_max, y_max, _ = self.pose_min + self.widths*(index + 1) #右上の座標（斜め上の離散状態の左下の座標）
        
        corners = [[x_min, y_min, _], [x_min, y_max, _], [x_max, y_min, _], [x_max, y_max, _] ] #4隅の座標
        return all([self.goal.inside(np.array(c).T) for c in corners ])
    
    def write_policy(self): ###dp2writepolicy
        with open("policy.txt", "w") as f:
            for index in self.indexes:
                p = self.policy[index]
                f.write("{} {} {} {} {}\n".format(index[0], index[1], index[2], p[0], p[1]))

In [3]:
import seaborn as sns   ###dp2exec

puddles = [Puddle((-2, 0), (0, 2), 0.1), Puddle((-0.5, -2), (2.5, 1), 0.1)] 

dp = DynamicProgramming(np.array([0.2, 0.2, math.pi/18]).T, Goal(-3,-3), puddles, 0.1, 10) 

counter = 0 #スイープの回数

In [4]:
delta = 1e100

while delta > 0.01: 
    delta = dp.value_iteration_sweep()
    counter += 1
    print(counter, delta)

1 10.7018316458
2 9.32259616331
3 8.18093180071
4 7.21520459786
5 6.36238804991
6 5.69304605969
7 5.44881608805
8 5.07771097381
9 4.80011661781
10 4.7526878394
11 4.58182293499
12 4.32562964546
13 4.25455603871
14 4.18292300218
15 4.03125128581
16 3.87580208434
17 3.85933631595
18 3.77292971199
19 3.63075644552
20 3.5916829519
21 3.5482071748
22 3.45394422955
23 3.37196979941
24 3.35186397412
25 3.29249765953
26 3.24720108091
27 3.18861169211
28 3.14569747074
29 3.12931150766
30 3.09110607669
31 3.02868339399
32 3.01850624204
33 2.99671171768
34 2.95161739259
35 2.91456867105
36 2.90608884114
37 2.87544014767
38 2.82509179106
39 2.81858209665
40 2.80030885414
41 2.76297404932
42 2.73451472136
43 2.72679221258
44 2.70075642182
45 2.65823004098
46 2.65540397675
47 2.63913564068
48 2.60688377186
49 2.56030924801
50 2.50526565274
51 2.50403889283
52 2.48221659821
53 2.44329738996
54 2.39026163395
55 2.32570452748
56 2.29991273286
57 2.29080893726
58 2.27224405121
59 2.24502598368
60 2.2100

449 0.538566521545
450 0.537624554997
451 0.536553251029
452 0.535356346039
453 0.534037433301
454 0.53259998614
455 0.531047375695
456 0.529617188983
457 0.528944918622
458 0.528916346438
459 0.528666153064
460 0.528180075452
461 0.527480861995
462 0.526587975047
463 0.525518061117
464 0.524285369962
465 0.522902123495
466 0.521378837094
467 0.52016738146
468 0.519356679848
469 0.518386408749
470 0.517271168688
471 0.516022209037
472 0.51520227341
473 0.514566379085
474 0.513798899362
475 0.513034499691
476 0.512173194779
477 0.511155402737
478 0.509996026701
479 0.508966977461
480 0.508358397983
481 0.507917834609
482 0.507409258533
483 0.506761950757
484 0.506100012364
485 0.50536275387
486 0.504460797722
487 0.503411987631
488 0.502229742755
489 0.501501907546
490 0.500918249741
491 0.500251467335
492 0.499830096846
493 0.499250223124
494 0.498527906434
495 0.497703288373
496 0.496771739257
497 0.49570802116
498 0.494602329183
499 0.494134934055
500 0.493608356768
501 0.49294715458

In [5]:
dp.write_policy()